<a href="https://colab.research.google.com/github/ArpanSM/MachineLearningHackathons/blob/master/Novartis_Data_Science_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/novartis-data-science-hiring-challenge/predict-if-the-server-will-be-hacked-a1889487/6a62a5b4967411ea.zip
!unzip 6a62a5b4967411ea.zip

--2020-06-13 09:01:27--  https://he-s3.s3.amazonaws.com/media/hackathon/novartis-data-science-hiring-challenge/predict-if-the-server-will-be-hacked-a1889487/6a62a5b4967411ea.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.32.216
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.32.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 555520 (542K) [application/zip]
Saving to: ‘6a62a5b4967411ea.zip’

6a62a5b4967411ea.zi 100%[===================>] 542.50K   493KB/s    in 1.1s    

2020-06-13 09:01:29 (493 KB/s) - ‘6a62a5b4967411ea.zip’ saved [555520/555520]

Archive:  6a62a5b4967411ea.zip
   creating: Dataset/
  inflating: Dataset/Train.csv       
  inflating: Dataset/sample_submission.csv  
  inflating: Dataset/Test.csv        


In [ ]:
!pip install h2o
import h2o
h2o.init()

     |████████████████████████████████| 128.3MB 54kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.4-py2.py3-none-any.whl size=128397038 sha256=b7d4a9abb7da5b6a7f61a4f249d5fcb0b29ee02b59fdff3b89e2807680487377
  Stored in directory: /root/.cache/pip/wheels/ad/bd/cc/0a8c7dffc4772df0bbfae4c1c7a9d64525fcd3ada9694dd8c3
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpp3cwxtlf
  JVM stdout: /tmp/tmpp3cwxtlf/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpp3cwxtlf/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Conne

H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_o8v5fg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
df = h2o.import_file('/content/Dataset/Train.csv')
dft = h2o.import_file('/content/Dataset/Test.csv')
df['MULTIPLE_OFFENSE'] = df['MULTIPLE_OFFENSE'].asfactor()
print(df.types)
print(dft.types)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
{'INCIDENT_ID': 'string', 'DATE': 'time', 'X_1': 'int', 'X_2': 'int', 'X_3': 'int', 'X_4': 'int', 'X_5': 'int', 'X_6': 'int', 'X_7': 'int', 'X_8': 'int', 'X_9': 'int', 'X_10': 'int', 'X_11': 'int', 'X_12': 'int', 'X_13': 'int', 'X_14': 'int', 'X_15': 'int', 'MULTIPLE_OFFENSE': 'enum'}
{'INCIDENT_ID': 'string', 'DATE': 'time', 'X_1': 'int', 'X_2': 'int', 'X_3': 'int', 'X_4': 'int', 'X_5': 'int', 'X_6': 'int', 'X_7': 'int', 'X_8': 'int', 'X_9': 'int', 'X_10': 'int', 'X_11': 'int', 'X_12': 'int', 'X_13': 'int', 'X_14': 'int', 'X_15': 'int'}


In [ ]:
df_train, df_val, df_test = df.split_frame(ratios=[.7,.15])
y = "MULTIPLE_OFFENSE"
x = df.columns
x.remove(y)
ignored_col_list = "INCIDENT_ID"
#x.remove(ignored_col_list)

In [ ]:
from h2o.automl import H2OAutoML
aml = H2OAutoML(nfolds=15,
                balance_classes=False,
                max_runtime_secs=240,
                max_models=0,
                exclude_algos=['DeepLearning','GLM'],
                verbosity = "info",
                sort_metric='AUC',
                stopping_metric='AUC')
aml.train(x=x,y=y,training_frame=df_train,validation_frame=df_val)
aml.leaderboard

AutoML progress: |
09:39:05.265: Project: AutoML_20200613_93905265
09:39:05.265: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
09:39:05.265: Setting stopping tolerance adaptively based on the training frame: 0.007716315844839853
09:39:05.265: Build control seed: -1 (random)
09:39:05.265: training frame: Frame key: automl_training_py_538_sid_968a    cols: 18    rows: 16795  chunks: 8    size: 644143  checksum: -6945373353730415299
09:39:05.266: validation frame: Frame key: py_539_sid_968a    cols: 18    rows: 3507  chunks: 8    size: 144880  checksum: 316101117747267471
09:39:05.266: leaderboard frame: NULL
09:39:05.266: blending frame: NULL
09:39:05.266: response column: MULTIPLE_OFFENSE
09:39:05.266: fold column: null
09:39:05.266: weights column: null
09:39:05.266: Loading ex

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_grid__1_AutoML_20200613_093905_model_1,0.999808,0.00832956,0.999991,0.00731009,0.040723,0.00165837
XGBoost_grid__1_AutoML_20200613_093905_model_4,0.999783,0.00740874,0.99999,0.0107472,0.0409587,0.00167762
StackedEnsemble_AllModels_AutoML_20200613_093905,0.999749,0.00697701,0.999988,0.00935369,0.0395261,0.00156231
StackedEnsemble_BestOfFamily_AutoML_20200613_093905,0.999697,0.006919,0.999986,0.00724783,0.0398018,0.00158418
XGBoost_grid__1_AutoML_20200613_093905_model_5,0.999608,0.0159777,0.999982,0.0194471,0.0507228,0.0025728
GBM_3_AutoML_20200613_093905,0.99947,0.034742,0.999975,0.02369,0.0765756,0.00586383
GBM_1_AutoML_20200613_093905,0.999466,0.0298763,0.999975,0.011,0.0726769,0.00528193
XGBoost_3_AutoML_20200613_093905,0.999464,0.0128156,0.999975,0.0148033,0.049382,0.00243859
XGBoost_1_AutoML_20200613_093905,0.999362,0.0191218,0.99997,0.0259515,0.0584844,0.00342042
XGBoost_grid__1_AutoML_20200613_093905_model_2,0.999342,0.00996801,0.999969,0.020035,0.0465005,0.00216229


In [ ]:
pred = aml.leader.predict(df_test)
aml.leader.model_performance(df_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%

ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.0021817980877095354
RMSE: 0.04670972155461361
LogLoss: 0.009876134060017376
Mean Per-Class Error: 0.0034886525100048527
AUC: 0.9997009959198572
AUCPR: 0.9999836279415495
Gini: 0.9994019918397143

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7473490834236145: 


,,0,1,Error,Rate
0,0,180.0,2.0,0.011,(2.0/182.0)
1,1,4.0,3368.0,0.0012,(4.0/3372.0)
2,Total,184.0,3370.0,0.0017,(6.0/3554.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.747349,0.999110,246.0
1,max f2,0.523497,0.999170,252.0
2,max f0point5,0.790477,0.999466,244.0
3,max accuracy,0.790477,0.998312,244.0
4,max precision,0.999877,1.000000,0.0
5,max recall,0.169197,1.000000,272.0
6,max specificity,0.999877,1.000000,0.0
7,max absolute_mcc,0.790477,0.982870,244.0
8,max min_per_class_accuracy,0.915129,0.994505,231.0
9,max mean_per_class_accuracy,0.790477,0.996511,244.0



Gains/Lift Table: Avg response rate: 94.88 %, avg score: 95.06 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.019696,0.999878,1.053974,1.053974,1.000000,0.999879,1.00000,0.999879,0.020759,0.020759,5.397390,5.39739
1,,2,0.036016,0.999878,1.053974,1.053974,1.000000,0.999878,1.00000,0.999878,0.017200,0.037960,5.397390,5.39739
2,,3,0.042206,0.999875,1.053974,1.053974,1.000000,0.999875,1.00000,0.999878,0.006524,0.044484,5.397390,5.39739
3,,4,0.050366,0.999861,1.053974,1.053974,1.000000,0.999871,1.00000,0.999877,0.008600,0.053084,5.397390,5.39739
4,,5,0.102983,0.999853,1.053974,1.053974,1.000000,0.999857,1.00000,0.999867,0.055457,0.108541,5.397390,5.39739
5,,6,0.154474,0.999845,1.053974,1.053974,1.000000,0.999849,1.00000,0.999861,0.054270,0.162811,5.397390,5.39739
6,,7,0.205121,0.999845,1.053974,1.053974,1.000000,0.999845,1.00000,0.999857,0.053381,0.216192,5.397390,5.39739
7,,8,0.300506,0.999832,1.053974,1.053974,1.000000,0.999842,1.00000,0.999852,0.100534,0.316726,5.397390,5.39739
8,,9,0.400113,0.999794,1.053974,1.053974,1.000000,0.999814,1.00000,0.999843,0.104982,0.421708,5.397390,5.39739
9,,10,0.502251,0.999770,1.053974,1.053974,1.000000,0.999786,1.00000,0.999831,0.107651,0.529359,5.397390,5.39739


In [ ]:
dft['predict'] = aml.leader.predict(dft)
h2o.download_csv(dft, "dft.csv")

xgboost prediction progress: |████████████████████████████████████████████| 100%


H2OResponseError: ignored

In [ ]:
import pandas as pd
sub = pd.read_csv('/content/Dataset/sample_submission.csv')
sub.head(20)

,INCIDENT_ID,MULTIPLE_OFFENSE
0,CR_195453,1
1,CR_103520,0
2,.,.
3,.,.
4,CR_196089,1
5,CR_112195,0
6,CR_149832,1
